This notebook is a basic example for applying salting method to study ambience interference.

In [ ]:
import saltax
import straxen
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

straxen.print_versions()

In [ ]:
rn_runs = ['022510','022735','022723','022717']#,'022685'] bad run for match?
bk_runs = ['026577','028031','031147','027705','025579','030079','029139','028047']

In [ ]:
st_salt = saltax.contexts.sxenonnt(runid=21952,
             saltax_mode='salt',
             output_folder='/project/lgrandi/yuanlq/salt',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')
st_simu = saltax.contexts.sxenonnt(runid=21952,
             saltax_mode='simu',
             output_folder='/project/lgrandi/yuanlq/salt',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')
st_data = saltax.contexts.sxenonnt(runid=21952,
             saltax_mode='data',
             output_folder='/project/lgrandi/yuanlq/salt',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')

In [ ]:
for i, run in enumerate(rn_runs):
    events_simu_i = st_simu.get_array(run, 'event_info')
    events_salt_i = st_salt.get_array(run, 'event_info')
    events_data_i = st_data.get_array(run, 'event_info')
    truth_i = st_simu.get_array(run, 'truth')
    match_i = st_simu.get_array(run, 'match_acceptance_extended')
    events_salt_matched_to_simu_i, \
        events_simu_matched_to_salt_i = saltax.match(truth_i[(truth_i['z']<-13)&(truth_i['z']>-145)&
                                                             (np.sqrt(truth_i['x']**2 + truth_i['y']**2)<63)], 
                                                     match_i[(truth_i['z']<-13)&(truth_i['z']>-145)&
                                                             (np.sqrt(truth_i['x']**2 + truth_i['y']**2)<63)],
                                                     events_simu_i[(events_simu_i['z']<-13)&(events_simu_i['z']>-145)], 
                                                     events_salt_i[(events_salt_i['z']<-13)&(events_salt_i['z']>-145)])
    if i==0:
        events_simu = events_simu_i
        events_salt = events_salt_i
        events_data = events_data_i
        truth = truth_i
        match = match_i
        events_salt_matched_to_simu = events_salt_matched_to_simu_i
        events_simu_matched_to_salt = events_simu_matched_to_salt_i
    else:
        events_simu = np.concatenate((events_simu,events_simu_i))
        events_salt = np.concatenate((events_salt,events_salt_i))
        events_data = np.concatenate((events_data,events_data_i))
        truth = np.concatenate((truth,truth_i))
        match = np.concatenate((match,match_i))
        events_salt_matched_to_simu = np.concatenate((events_salt_matched_to_simu,
                                                      events_salt_matched_to_simu_i))
        events_simu_matched_to_salt = np.concatenate((events_simu_matched_to_salt,
                                                      events_simu_matched_to_salt_i))

In [ ]:
truth_s1 = []
truth_s2 = []
for i in range(np.max(truth_i['event_number'])):
    selected_truth = truth_i[truth_i['event_number']==i]
    if len(selected_truth) == 2 and len(np.unique(selected_truth['type']))==2:
        if np.sqrt(selected_truth['x']**2 + selected_truth['y']**2)[0] <= 63:
            truth_s1.append(selected_truth[selected_truth['type']==1][0]['raw_area_trigger'])
            truth_s2.append(selected_truth[selected_truth['type']==2][0]['raw_area_trigger'])

plt.scatter(truth_s1, truth_s2, s=1)
plt.xlim(0,150)
plt.ylim(0,6000)
plt.xlabel('True S1 [PE]')
plt.ylabel('True S2 [PE]')
plt.title('WFSim Truth ER Band')

In [ ]:
for i, run in enumerate(bk_runs):
    events_simu_i = st_simu.get_array(run, 'event_info')
    events_salt_i = st_salt.get_array(run, 'event_info')
    events_data_i = st_data.get_array(run, 'event_info')
    truth_i = st_simu.get_array(run, 'truth')
    match_i = st_simu.get_array(run, 'match_acceptance_extended')
    events_salt_matched_to_simu_i, \
        events_simu_matched_to_salt_i = saltax.match(truth_i[(truth_i['z']<-13)&(truth_i['z']>-145)], 
                                                     match_i[(truth_i['z']<-13)&(truth_i['z']>-145)],
                                                     events_simu_i[(events_simu_i['z']<-13)&(events_simu_i['z']>-145)], 
                                                     events_salt_i[(events_salt_i['z']<-13)&(events_salt_i['z']>-145)])
    if i == 0:
        events_simub = events_simu_i
        events_saltb = events_salt_i
        events_datab = events_data_i
        truthb = truth_i
        matchb = match_i
        events_salt_matched_to_simub = events_salt_matched_to_simu_i
        events_simu_matched_to_saltb = events_simu_matched_to_salt_i
    else:
        events_simub = np.concatenate((events_simub,events_simu_i))
        events_saltb = np.concatenate((events_saltb,events_salt_i))
        events_datab = np.concatenate((events_datab,events_data_i))
        truthb = np.concatenate((truthb,truth_i))
        matchb = np.concatenate((matchb,match_i))
        events_salt_matched_to_simub = np.concatenate((events_salt_matched_to_simub,
                                                      events_salt_matched_to_simu_i))
        events_simu_matched_to_saltb = np.concatenate((events_simu_matched_to_saltb,
                                                      events_simu_matched_to_salt_i))

In [ ]:
plt.figure(dpi=150)
plt.scatter(events_simu['cs1'],events_simu['cs2'],s=1, label='Salted')
plt.scatter(events_data['cs1'],events_data['cs2'],s=1, label='Data')
plt.legend()
plt.xlim(0,100)
plt.ylim(0,6000)
plt.xlabel('CS1 [PE]')
plt.ylabel('CS2 [PE]')
plt.title('Salted Rn220 into SR0 Rn220 Calibration')

In [ ]:
n_bins = 31
cs1_bins = np.linspace(0,100,n_bins)
salt_med = []
simu_med = []
salt_1sig_u = []
simu_1sig_u = []
salt_1sig_l = []
simu_1sig_l = []
salt_2sig_u = []
simu_2sig_u = []
salt_2sig_l = []
simu_2sig_l = []

for i in range(n_bins-1):
    selected_salt = events_salt_matched_to_simu[(events_salt_matched_to_simu['cs1']>=cs1_bins[i])*
                                                (events_salt_matched_to_simu['cs1']<cs1_bins[i+1])]
    salt_med.append(np.median(selected_salt['cs2'][selected_salt['cs2']>0]))
    salt_1sig_l.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 16.5))
    salt_1sig_u.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 83.5))
    salt_2sig_l.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 2.5))
    salt_2sig_u.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 97.5))
    
    selected_simu = events_simu_matched_to_salt[(events_simu_matched_to_salt['cs1']>=cs1_bins[i])*
                                                (events_simu_matched_to_salt['cs1']<cs1_bins[i+1])]
    simu_med.append(np.median(selected_simu['cs2'][selected_simu['cs2']>0]))
    simu_1sig_l.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 16.5))
    simu_1sig_u.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 83.5))
    simu_2sig_l.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 2.5))
    simu_2sig_u.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 97.5))

cs1_coord = np.linspace(0,100,n_bins-1) + 100/2/(n_bins-1)

plt.figure(dpi=150)
plt.scatter(events_salt_matched_to_simu['cs1'],events_salt_matched_to_simu['cs2'],s=0.5, label='Salted', alpha=0.1)
plt.scatter(events_simu_matched_to_salt['cs1'],events_simu_matched_to_salt['cs2'],s=0.5, label='Simulated', alpha=0.1)
plt.plot(cs1_coord, salt_med, color='b', label='Salt Median')
plt.plot(cs1_coord, simu_med, color='r', label='Simu Median')
plt.plot(cs1_coord, salt_1sig_l, color='b', label='Salt 1sig',linestyle='dashed')
plt.plot(cs1_coord, salt_1sig_u, color='b', linestyle='dashed')
plt.plot(cs1_coord, salt_2sig_l, color='b', label='Salt 2sig',linestyle='dashed', alpha=0.5)
plt.plot(cs1_coord, salt_2sig_u, color='b', linestyle='dashed', alpha=0.5)
plt.plot(cs1_coord, simu_1sig_l, color='r', label='Simu 1sig',linestyle='dashed')
plt.plot(cs1_coord, simu_1sig_u, color='r', linestyle='dashed')
plt.plot(cs1_coord, simu_2sig_l, color='r', label='Simu 2sig',linestyle='dashed', alpha=0.5)
plt.plot(cs1_coord, simu_2sig_u, color='r', linestyle='dashed', alpha=0.5)


plt.legend()
plt.xlim(0,100)
plt.ylim(0,6000)
plt.xlabel('CS1 [PE]')
plt.ylabel('CS2 [PE]')
plt.title('Ambience Interference in SR0 Rn220')

In [ ]:
plt.figure(dpi=150)
plt.scatter(events_simu_matched_to_salt['cs1'],events_salt_matched_to_simu['cs1'], s=1, label='Rn220')
plt.scatter(events_simu_matched_to_saltb['cs1'],events_salt_matched_to_simub['cs1'], s=1, label='BKG')
plt.legend()
plt.xlim(0,150)
plt.ylim(0,150)
plt.title('Salted Rn220 into SR0 Rn220/BKG')
plt.xlabel('Simu CS1 [PE]')
plt.ylabel('Salt CS1 [PE]')

In [ ]:
plt.figure(dpi=150)
plt.scatter(events_simu_matched_to_salt['cs2'],events_salt_matched_to_simu['cs2'], s=1, label='Rn220')
plt.scatter(events_simu_matched_to_saltb['cs2'],events_salt_matched_to_simub['cs2'], s=1, label='BKG')
plt.legend()
plt.xlim(0,6000)
plt.ylim(0,6000)
plt.title('Salted Rn220 into SR0 Rn220/BKG')
plt.xlabel('Simu CS2 [PE]')
plt.ylabel('Salt CS2 [PE]')

In [ ]:
plt.figure(dpi=150)
plt.hist(events_salt_matched_to_simu['cs1'] - events_simu_matched_to_salt['cs1'], 
         bins=np.linspace(-200,200,50), alpha=0.5, label='Rn220', density=True)
plt.hist(events_salt_matched_to_simub['cs1'] - events_simu_matched_to_saltb['cs1'], 
         bins=np.linspace(-200,200,50), alpha=0.5, label='BKG', density=True)
plt.legend()
plt.title('Salted Rn220 into SR0 Rn220/BKG')
plt.xlabel('Ambience Contribution in CS1 [PE]')
plt.yscale('log')

In [ ]:
plt.figure(dpi=150)
plt.hist(events_salt_matched_to_simu['cs2'] - events_simu_matched_to_salt['cs2'], 
         bins=np.linspace(-200,200,50), alpha=0.5, label='Rn220', density=True)
plt.hist(events_salt_matched_to_simub['cs2'] - events_simu_matched_to_saltb['cs2'], 
         bins=np.linspace(-200,200,50), alpha=0.5, label='BKG', density=True)
plt.title('Salted Rn220 into SR0 Rn220/BKG')
plt.legend()
plt.xlabel('Ambience Contribution in CS2 [PE]')
plt.yscale('log')

In [ ]:
inds_big_change_in_s2 = np.where((events_salt_matched_to_simu['cs2'] - events_simu_matched_to_salt['cs2']>100)&
                                 (events_salt_matched_to_simu['cs2'] - events_simu_matched_to_salt['cs2']<200))[0]

In [ ]:
saltax.plot_wf(ind=inds_big_change_in_s2[7], st_salt=st_salt, st_simu=st_simu, st_data=st_data, 
               runid='022685', matched_simu=events_simu_matched_to_salt, 
               event_ext_window_ns=2.4e6, 
               s1_ext_window_samples=25, 
               s2_ext_window_samples=100)